In [ ]:
import requests
import json
import time
import random
import numpy as np
#from numpy import linalg as la 
#from numpy import *
from sklearn.metrics.pairwise import pairwise_distances

In [ ]:
time.localtime(1500000000)[:2]

## 整理数据
### 划分训练集和测试集，按照2017.7划分

In [ ]:
invb={};inva={}
#读取所有的投资关系
with open("data/projects.json",encoding='utf8') as f:
    dic=json.loads(f.read())

#整理投资关系，2017.7前后都有投资的，加入invb,inva
count=0
for k in dic.keys():
    #print(count,k);count+=1
    #记录当前的，根据情况判断是否加入inva，invb
    bn=[];an=[]
    investments=dic[k]["investments"]
    if len(investments)<2:
        continue
    for inv in investments:
        tag=True
        #对一家企业可能投过多次，看是否在时间节点之前投资过
        for invn in inv["investments"]:
            if invn['investAt']<1500000000000:
                bn.append(inv['project']['companyName'])
                tag=False
                break
        if tag:
            an.append(inv['project']['companyName'])
    if len(an)>0 and len(bn)>1:
        invb[dic[k]['org']['nameAbbr']]=bn
        inva[dic[k]['org']['nameAbbr']]=an

In [ ]:
invb={};inva={}
#读取所有的投资关系
with open("data/projects.json",encoding='utf8') as f:
    dic=json.loads(f.read())

#整理投资关系，2017.7前后都有投资的，加入invb,inva
count=0
for k in dic.keys():
    #print(count,k);count+=1
    #记录当前的，根据情况判断是否加入inva，invb
    bn=[];an=[]
    investments=dic[k]["investments"]
    if len(investments)<2:
        continue
    for inv in investments:
        #对一家企业可能投过多次，看是否在时间节点之前投资过
        for invn in inv["investments"]:
            if invn['investAt']<1500000000000:
                bn.append(inv['project']['companyName'])
                break
        for invn in inv["investments"]:
            if invn['investAt']>=1500000000000:
                an.append(inv['project']['companyName'])
                break
    if len(an)>0 and len(bn)>1:
        invb[dic[k]['org']['nameAbbr']]=bn
        inva[dic[k]['org']['nameAbbr']]=an

In [ ]:
#将划分时间之后出现的企业中，没有在之前出现的去掉
comb=[];coma=[]
for k in inva:
    comb+=invb[k]
    coma+=inva[k]
print(len(comb),len(coma))
print(len(set(comb)),len(set(coma)))
setb=set(comb)
comafter=[]
for c in list(set(coma)):
    if c in setb:
        #print(c)
        comafter.append(c)
print(len(comafter))

In [ ]:
comdict={}#企业对应的顺序
comlist=[]
for i,c in enumerate(sorted(list(set(comb)))):
    comdict[c]=i
    comlist.append(c)
#投资机构列表
investors=sorted(list(invb.keys()))
print('投资机构数量',len(investors))
#投资关系矩阵
inv_matrix=np.zeros((len(investors),len(set(comb))))
for i,inv in enumerate(investors):
    for c in invb[inv]:
        #print(i,comdict[c])
        inv_matrix[i,comdict[c]]=1.0
#物品的相似矩阵
item_similarity = pairwise_distances(inv_matrix.T, metric = "cosine")

In [ ]:
comnot=[]
for c in comdict:
    if c not in comafter:
        comnot.append(comdict[c])

In [ ]:
#推荐列表
def recommend(user,dataMat,simMat):
    invested=[]#投资过的企业列表
    for i in range(dataMat.shape[1]):
        if dataMat[user,i]!=0:
            invested.append(i)
    mean_vec=np.sum(1-simMat[invested],axis=0)/len(invested)
    #mean_vec[invested]=-1
    mean_vec[comnot]=-1
    ratelist=[[i,rate] for i,rate in enumerate(mean_vec)]
    random.shuffle(ratelist)
    return sorted(ratelist,key=lambda k:k[1],reverse=True)
    #return [com[0] for com in sorted(ratelist,key=lambda k:k[1],reverse=True)]

In [ ]:
c10=c100=c500=allcount=0
for i,investor in enumerate(investors):
    recres=recommend(i,inv_matrix,item_similarity)
    if recres[0][1]==0:
        continue
    reclist=[com[0] for com in recres]
    reallist=inva[investor]
    for r in reallist:
        if r in comafter:
            allcount+=1
            if comdict[r] in reclist[:10]:
                c10+=1;c100+=1;c500+=1
            elif comdict[r] in reclist[:100]:
                c100+=1;c500+=1
            elif comdict[r] in reclist[:500]:
                c500+=1
    if allcount>0:
        print(i,c10/allcount,c100/allcount,c500/allcount)

In [ ]:
recommend(2000,inv_matrix,item_similarity)

In [ ]:
investors

In [ ]:
item_similarity.shape[0]**2-np.sum(item_similarity)

In [ ]:
np.sum(inv_matrix)/(inv_matrix.shape[0]*inv_matrix.shape[1])

In [ ]:
'Yotpo' in comafter

In [ ]:
item_similarity.shape

In [ ]:
comb[22141]

In [ ]:
np.sum(inv_matrix,axis=1)

In [ ]:
np.sum(inv_matrix[51])